<a href="https://colab.research.google.com/github/Krishnanand-G/Aicte_Internship_Project/blob/main/DNN_ImageSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# Copy the zip file from Google Drive
!cp "/content/drive/My Drive/dataset.zip" "/content/dataset.zip"

In [ ]:
# Define zip and extraction paths
import zipfile
zip_path = "/content/dataset.zip"
extract_path = "/content/dataset"

In [ ]:
# Extract the zip file
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Extraction successful!")
except zipfile.BadZipFile:
    print("Error: The file is not a valid ZIP archive.")

In [ ]:
# Check the extracted folders
import os
dataset_path = "/content/dataset"
print("Folders in dataset:", os.listdir(dataset_path))

In [ ]:
import os
print("Root contents:", os.listdir("/content/dataset"))
print("Data contents:", os.listdir("/content/dataset/data"))

Root contents: ['data']
Data contents: ['dogs', 'bike', 'flowers', 'cats', 'cars', 'horses', 'human', 'data']


In [ ]:
# Dataset is likely inside a subfolder (e.g., 'data'), update path accordingly
dataset_path = "/content/dataset/data/data"

In [ ]:
# Load dataset using ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (28, 28)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=False,
    fill_mode='nearest'
)

In [ ]:
train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

Found 1444 images belonging to 7 classes.


In [ ]:
val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

Found 359 images belonging to 7 classes.


In [ ]:
# Build the model for 7-class classification
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
model = Sequential([
    Flatten(input_shape=(28, 28, 1)),
    BatchNormalization(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(7, activation='softmax')  # Change 8 to 7 if you have 7 classes
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

In [ ]:
model.fit(train_data, validation_data=val_data, epochs=20, callbacks=[early_stopping, reduce_lr])

Epoch 1/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 6s 88ms/step - accuracy: 0.2366 - loss: 2.3603 - val_accuracy: 0.2535 - val_loss: 1.8899 - learning_rate: 5.0000e-04
Epoch 2/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 5s 106ms/step - accuracy: 0.3386 - loss: 1.8801 - val_accuracy: 0.2786 - val_loss: 1.9229 - learning_rate: 5.0000e-04
Epoch 3/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 4s 79ms/step - accuracy: 0.3707 - loss: 1.7469 - val_accuracy: 0.3092 - val_loss: 1.7466 - learning_rate: 5.0000e-04
Epoch 4/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 4s 79ms/step - accuracy: 0.3992 - loss: 1.6817 - val_accuracy: 0.3593 - val_loss: 1.6705 - learning_rate: 5.0000e-04
Epoch 5/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 5s 100ms/step - accuracy: 0.4076 - loss: 1.6369 - val_accuracy: 0.3705 - val_loss: 1.6363 - learning_rate: 5.0000e-04
Epoch 6/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 4s 82ms/step - accuracy: 0.4092 - loss: 1.6573 - val_accuracy: 0.3816 - val_loss: 1.6247 - learning_rate: 5.0000e-04
Epoch 7/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 4s 78ms/step - accuracy: 0.442

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(val_data)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.4009 - loss: 1.5398
Validation Loss: 1.6160
Validation Accuracy: 0.3733


In [ ]:
# Predict a single image
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
mg_path = "/content/test.jpg"

img = image.load_img(img_path, target_size=(28, 28), color_mode='grayscale')
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

In [ ]:
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions[0])

class_labels = train_data.class_indices
labels = dict((v, k) for k, v in class_labels.items())

print(f"Predicted class: {labels[predicted_class]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Predicted class: horses
